In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

In [7]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url,root='.')

100%|██████████| 135107811/135107811 [00:02<00:00, 50873907.27it/s]


In [8]:
with tarfile.open('/kaggle/working/cifar10.tgz') as trf:
    trf.extractall('./data')

In [11]:
classes = dict(enumerate(os.listdir('/kaggle/working/data/cifar10/train')))

In [12]:
classes

{0: 'frog',
 1: 'cat',
 2: 'automobile',
 3: 'ship',
 4: 'dog',
 5: 'horse',
 6: 'deer',
 7: 'airplane',
 8: 'bird',
 9: 'truck'}

In [22]:
img = os.listdir('/kaggle/working/data/cifar10/train/frog')[0]

In [28]:
from PIL import Image
img_ = Image.open('/kaggle/working/data/cifar10/train/frog/'+img)

In [29]:
img_.size

(32, 32)

In [20]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms,ToTensor,RandomRotation

In [32]:
transform = transforms.Compose((
    RandomRotation(13),
    ToTensor()
)
)

In [48]:
train_dataset = ImageFolder('/kaggle/working/data/cifar10/train',transform=transform)
test_dataset = ImageFolder('/kaggle/working/data/cifar10/test',transform=transform)

In [56]:
train_dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [57]:
test_dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [ ]:
from torch import nn

In [ ]:
nn.ReLU()

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,activation_function):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation_function
        self.kernel_size = kernel_size
        self.block = nn.ModuleList([
            nn.Conv2d(in_channels = self.in_channels,
                      out_channels= self.out_channels,
                      padding=1,
                      kernel_size=self.kernel_size,
                      stride=1,
                      bias=False),
            self.activation(True),
            nn.BatchNorm2d(self.out_channels),
            nn.Conv2d(in_channels = self.out_channels,
                      out_channels= self.out_channels,
                      kernel_size=1,
                      stride=1,
                      bias=False),
            self.activation(True),
            nn.BatchNorm2d(self.out_channels),
        ])
        
    def forward(self,X):
        out = X
        for layer in self.block:
            out = layer(out)
        print(out.shape)
        print(X.shape)
        return X +out

In [ ]:
class ResNet(nn.Module):
    def __init__(self,in_channels, num_classes):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.filters = 64
        
        self.con1 = nn.Conv2d(in_channels=3,
                             out_channels=self.filters**1,
                             kernel_size=7,
                             stride=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.block1 = nn.Sequential(
                        ResNetBlock(self.filters**1,self.filters**1,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**1,self.filters**1,kernel_size=3,activation_function=nn.ReLU),
        )
        
        self.conv2 = nn.Conv2d(in_channels=self.filters**1,out_channels=self.filters**2,kernel_size=3,stride=2)        
        self.block2 = nn.Sequential(
                        ResNetBlock(self.filters**2,self.filters**2,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**2,self.filters**2,kernel_size=3,activation_function=nn.ReLU),
        )
        
        self.conv3 = nn.Conv2d(in_channels=self.filters**2,out_channels=self.filters**3,kernel_size=3,stride=2)        
        self.block3 = nn.Sequential(
                        ResNetBlock(self.filters**3,self.filters**3,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**3,self.filters**3,kernel_size=3,activation_function=nn.ReLU),
        )
    def forward(x):
        x = self.co1(x)
        x = self.maxpool1(x)
        x = self.block1(x)
        x = self.conv2(x)
        x = self.block2(x)
        x = self.conv3(x)
        x = self.block3(x)
        print(x.shape)
        
        return x

In [ ]:
ResNet(3,10)(random_sample)